In [ ]:
import glob
import pandas as pd
from pathlib import Path
import re

In [63]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

### Data Information

In [ ]:
root = Path('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/')

In [ ]:
!ls $root

In [25]:
!ls $root

00	       crops_dirty_manual   gall_dadetv4_all	   info
crops_confuse  crops_manual_return  gall_dadetv4_rev_last


In [ ]:
!ls $root/gall_dadetv4_all/crops4clear

In [22]:
root_org = root /'gall_dadetv4_all/crops4clear_ai'

In [ ]:
len(glob.glob(root_org /''))

In [ ]:
len(manual_df)

In [21]:
!ls $root_org.parent

crops		crops_dirty_ai	 crops_oss	   imgs_det
crops4clear_ai	crops_folder_ai  crops_oss.tar.gz  labels


#### 每个类counter

In [ ]:
# folder_name = 'bag'
org_list = glob.glob(str(root_org /f'*/*/*/*.jpg'))
manual_list = glob.glob(str(root /f'crops_manual_return/*/*/*/*.jpg'))
len(manual_list)/len(org_list), len(manual_list), len(org_list)

In [ ]:
folder_name = 'bag'
org_list = glob.glob(str(root_org /f'{folder_name}/*/*/*.jpg'))
manual_list = glob.glob(str(root /f'crops_manual_return/{folder_name}/*/*/*.jpg'))
len(manual_list)/len(org_list), len(manual_list), len(org_list)

In [23]:
%%time
folder_name = 'lowerbody'
org_list = glob.glob(str(root_org /f'{folder_name}/*/*/*'))
manual_list = glob.glob(str(root /f'crops_manual_return/{folder_name}/*/*/*'))
len(manual_list)/len(org_list)

0.0

In [ ]:
folder_name = 'shoe'
org_list = glob.glob(str(root_org /f'{folder_name}/*/*/*'))
manual_list = glob.glob(str(root /f'crops_manual_return/{folder_name}/*/*/*'))
len(manual_list)/len(org_list)

In [ ]:
folder_name = 'skirt'
org_list = glob.glob(str(root_org /f'{folder_name}/*/*/*'))
manual_list = glob.glob(str(root /f'crops_manual_return/{folder_name}/*/*/*'))
len(manual_list)/len(org_list), len(manual_list),len(org_list)

In [28]:
%%time
folder_name = 'upperbody'
org_list = glob.glob(str(root_org /f'{folder_name}/*/*/*'))
manual_list = glob.glob(str(root /f'crops_manual_return/{folder_name}/*/*/*'))
len(manual_list)/len(org_list), len(manual_list), len(org_list)

CPU times: user 168 ms, sys: 194 ms, total: 363 ms
Wall time: 1.24 s


(0.9076865983207311, 19567, 21557)

In [ ]:
folder_name = 'wholebody'
org_list = glob.glob(str(root_org /f'{folder_name}/*/*/*'))
manual_list = glob.glob(str(root /f'crops_manual_return/{folder_name}/*/*/*'))
len(manual_list)/len(org_list), len(manual_list),len(org_list)

##### 总体acc

In [ ]:
name = ['bag',  'lowerbody', 'shoe',  'skirt', 'wholebody']
# name = ['bag',  'lowerbody', 'shoe',  'skirt',  'upperbody', 'wholebody']
o_l = 0
m_l = 0
for n in name:
    folder_name = n
    o_l += len(glob.glob(str(root_org /f'{folder_name}/*/*/*')))
    m_l += len(glob.glob(str(root /f'crops_manual_return/{folder_name}/*/*/*')))

In [ ]:
m_l /o_l

#### 反选手工挑选错误的

In [29]:
root_dirty = root /'crops_dirty_manual'

In [30]:
org_df = pd.DataFrame()
org_df['path'] = org_list
manual_df = pd.DataFrame()
manual_df['path'] = manual_list
org_df['stem'] = org_df['path'].apply(lambda x: Path(x).stem)
manual_df['stem'] = manual_df['path'].apply(lambda x: Path(x).stem)

In [31]:
dirty_df = org_df[~org_df['stem'].isin(manual_df['stem'])]

In [32]:
len(dirty_df)

1990

In [33]:
root_dirty

PosixPath('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/crops_dirty_manual')

In [34]:
from tqdm import tqdm
import shutil
for p in tqdm(dirty_df['path'].values):
    sp = Path(p)
    tp = Path('{0}/{1}/{2}/{3}'.format(root_dirty, sp.parts[-4], sp.parts[-3], sp.name))
    if not tp.parent.is_dir():
        tp.parent.mkdir(exist_ok=True, parents=True)
    shutil.copyfile(sp, tp)
    # break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1990/1990 [00:04<00:00, 469.75it/s]


#### bbox反查原图

In [56]:
!ls $root

00	       crops_dirty_manual   gall_dadetv4_all	   info
crops_confuse  crops_manual_return  gall_dadetv4_rev_last


In [103]:
!ls $root/'info/dadetv4'

bboxes_detv4_df.csv	  crops_all_df.csv  labels_df.csv
bboxes_detv4_info_df.csv  imgs_det_df.csv
bboxes_only_all_df.csv	  imgs_org_df.csv


In [74]:
root_dirty = root /'crops_dirty_manual'
!ls $root_dirty

bag  lowerbody	shoe  skirt  upperbody	wholebody


In [76]:
%%time
dirty_list = glob.glob(str(root_dirty /'*/*/*'))
len(dirty_list)

CPU times: user 22.7 ms, sys: 3.48 ms, total: 26.2 ms
Wall time: 36.7 ms


7845

In [79]:
dirty_all_df = pd.DataFrame()
dirty_all_df['dirty_sp'] = dirty_list

In [94]:
import re
dirty_all_df['stem'] = dirty_all_df['dirty_sp'].apply(lambda x: Path(x).stem)
dirty_all_df['stem'] = dirty_all_df['stem'].apply(lambda x: re.findall('(.*?)_[0-9]{1,2}', x)[0])

In [107]:
candidate_df = pd.read_csv(root/'info/dadetv4/imgs_org_df.csv')

In [109]:
stem2img_gallery = get_list2dict(candidate_df['stem'].values, candidate_df['oss_sp'].values)

In [110]:
len(stem2img_gallery)

1661397

In [111]:
dirty_all_df['oss_sp'] = dirty_all_df['stem'].map(stem2img_gallery)

In [115]:
len(dirty_all_df[~dirty_all_df['stem'].isin(candidate_df['stem'])])

0

In [117]:
dirty_all_df.head(1)

,dirty_sp,stem,oss_sp
0,/nas/chenyi/datasets_cls/gallery_detect/galler...,0cf3f012-121e-40cb-9844-5e4b76ca360b-829051,/nas/oss/gallery/0c/f3/0cf3f012-121e-40cb-9844...


##### det img.

In [118]:
!ls $root

00	       crops_dirty_manual   gall_dadetv4_all	   info
crops_confuse  crops_manual_return  gall_dadetv4_rev_last


In [122]:
!ls $root/'gall_dadetv4_all/'

crops		crops_dirty_ai	 crops_oss	   imgs_det
crops4clear_ai	crops_folder_ai  crops_oss.tar.gz  labels


In [123]:
det_root = root/'gall_dadetv4_all/imgs_det'

In [124]:
dirty_all_df['det_sp'] = dirty_all_df['stem'].apply(lambda x: f'{det_root}/{x}.jpg')

In [128]:
len(dirty_all_df), len(dirty_all_df.drop_duplicates('stem'))

(7845, 7320)

In [125]:
detp = dirty_all_df.iloc[0, 3]

In [129]:
from PIL import Image
# Image.open(detp)

In [133]:
dirty_all_df.head(1)

,dirty_sp,stem,oss_sp,det_sp
0,/nas/chenyi/datasets_cls/gallery_detect/galler...,0cf3f012-121e-40cb-9844-5e4b76ca360b-829051,/nas/oss/gallery/0c/f3/0cf3f012-121e-40cb-9844...,/nas/chenyi/datasets_cls/gallery_detect/galler...


In [137]:
dirty_all_df.head(1).values

array([['/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/crops_dirty_manual/wholebody/0c/0cf3f012-121e-40cb-9844-5e4b76ca360b-829051_4.jpg',
        '0cf3f012-121e-40cb-9844-5e4b76ca360b-829051',
        '/nas/oss/gallery/0c/f3/0cf3f012-121e-40cb-9844-5e4b76ca360b-829051.jpg',
        '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/imgs_det/0cf3f012-121e-40cb-9844-5e4b76ca360b-829051.jpg',
        '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/dirty_org_img/0c/0cf3f012-121e-40cb-9844-5e4b76ca360b-829051.jpg']],
      dtype=object)

In [136]:
dirty_org_root = root/'dirty_org_img'
dirty_all_df['img_tp'] = dirty_all_df['oss_sp'].apply(lambda x: '{0}/{1}/{2}'.format(dirty_org_root, Path(x).name[:2], Path(x).name))

In [140]:
import shutil
from tqdm import tqdm
for ind, row in tqdm(dirty_all_df.iterrows()):
    sp = Path(row['oss_sp'])
    tp = Path(row['img_tp'])
    if not tp.parent.is_dir():
        tp.parent.mkdir(parents=True, exist_ok=True)
    if sp.is_file() and not tp.is_file():
        shutil.copyfile(sp, tp)
    # break

7845it [03:06, 42.11it/s]
